# Analyzing CFR Agent Behavior

Why don't liberals perform better? Let's trace through games and see what each player does.

In [ ]:
import sys
sys.path.insert(0, '../..')

import pickle
from pathlib import Path
from collections import defaultdict

from shitler_env.game import ShitlerEnv
from agents.cfr.cfr_agent import CFRAgent, get_legal_actions
from agents.cfr.infoset import get_infoset_key, compute_suspicion_scores

In [ ]:
# Load the trained CFR agent
checkpoint_path = Path('checkpoints/cfr_iter_500000.pkl')
with open(checkpoint_path, 'rb') as f:
    data = pickle.load(f)

cfr_agent = CFRAgent()
cfr_agent.regret_sums = data['regret_sums']
cfr_agent.strategy_sums = data['strategy_sums']

print(f"Loaded {len(cfr_agent.regret_sums)} infosets")

## Play a game with verbose output

In [ ]:
ROLE_NAMES = {0: 'Liberal', 1: 'Fascist', 2: 'Hitler'}
PHASE_NAMES = {
    'nomination': 'Nominate Chancellor',
    'voting': 'Vote',
    'prez_cardsel': 'President Card Selection',
    'chanc_cardsel': 'Chancellor Card Selection', 
    'prez_claim': 'President Claim',
    'chanc_claim': 'Chancellor Claim',
    'execution': 'Execution'
}

def play_verbose_game(cfr_agent, seed=None):
    """Play a game and print every decision."""
    env = ShitlerEnv()
    if seed is not None:
        import random
        random.seed(seed)
    env.reset()
    
    # Print roles
    print("=" * 70)
    print("GAME START")
    print("=" * 70)
    roles = env.roles
    for i, role in enumerate(roles):
        print(f"  Player {i}: {ROLE_NAMES[role]}")
    print()
    
    turn = 0
    while not all(env.terminations.values()):
        current_agent = env.agent_selection
        current_idx = env.agents.index(current_agent)
        obs = env.observe(current_agent)
        phase = env.phase
        
        legal_actions = get_legal_actions(env, current_agent)
        if not legal_actions:
            env.step(0)
            continue
        
        # Get strategy
        infoset_key = get_infoset_key(obs, phase, current_idx)
        strategy = cfr_agent.get_average_strategy(infoset_key, legal_actions)
        
        # Sample action
        action = cfr_agent.sample_action(strategy)
        
        # Print decision
        role = ROLE_NAMES[roles[current_idx]]
        phase_name = PHASE_NAMES.get(phase, phase)
        
        print(f"Turn {turn}: Player {current_idx} ({role}) - {phase_name}")
        print(f"  Legal actions: {legal_actions}")
        print(f"  Strategy: {dict(sorted(strategy.items()))}")
        print(f"  Chose: {action}")
        
        # Add context for specific phases
        if phase == 'nomination':
            print(f"  (Nominating player {action} as Chancellor)")
        elif phase == 'voting':
            print(f"  ({'JA' if action == 1 else 'NEIN'})")
        elif phase in ['prez_cardsel', 'chanc_cardsel']:
            cards = obs.get('cards', [])
            print(f"  Cards in hand: {['Liberal' if c == 0 else 'Fascist' for c in cards]}")
            if action < len(cards):
                print(f"  Discarding: {'Liberal' if cards[action] == 0 else 'Fascist'}")
        elif phase in ['prez_claim', 'chanc_claim']:
            claim_map = {0: '0 fascist', 1: '1 fascist', 2: '2 fascist', 3: '3 fascist'}
            print(f"  Claiming: {claim_map.get(action, action)}")
        elif phase == 'execution':
            print(f"  (Executing player {action})")
        print()
        
        env.step(action)
        turn += 1
        
        # Print game state after voting rounds
        if phase == 'chanc_claim' or (phase == 'voting' and env.phase != 'voting'):
            print(f"  >> Liberal policies: {env.lib_policies}, Fascist policies: {env.fasc_policies}")
            print(f"  >> Election tracker: {env.election_tracker}")
            print()
    
    # Print result
    print("=" * 70)
    print("GAME END")
    print("=" * 70)
    
    # Determine winner
    lib_reward = env.rewards['player_0'] if roles[0] == 0 else env.rewards['player_1']
    if lib_reward > 0:
        print("LIBERALS WIN!")
    else:
        print("FASCISTS WIN!")
    
    print(f"Final: {env.lib_policies} liberal policies, {env.fasc_policies} fascist policies")
    print(f"Executed: {[i for i, e in enumerate(env.executed) if e]}")
    
    return env

In [ ]:
# Play a game
env = play_verbose_game(cfr_agent, seed=42)

## Analyze voting patterns

In [ ]:
def analyze_voting_strategies(cfr_agent, num_games=100):
    """Analyze how different roles vote."""
    
    # Track: role -> {ja_count, nein_count}
    vote_stats = {role: {'ja': 0, 'nein': 0, 'total': 0} for role in [0, 1, 2]}
    
    # Track: role voting for hitler_chancellor -> {ja, nein}
    hitler_chanc_votes = {role: {'ja': 0, 'nein': 0} for role in [0, 1, 2]}
    
    for game_idx in range(num_games):
        env = ShitlerEnv()
        env.reset()
        
        roles = env.roles
        hitler_idx = roles.index(2)
        
        while not all(env.terminations.values()):
            current_agent = env.agent_selection
            current_idx = env.agents.index(current_agent)
            obs = env.observe(current_agent)
            phase = env.phase
            
            legal_actions = get_legal_actions(env, current_agent)
            if not legal_actions:
                env.step(0)
                continue
            
            infoset_key = get_infoset_key(obs, phase, current_idx)
            strategy = cfr_agent.get_average_strategy(infoset_key, legal_actions)
            action = cfr_agent.sample_action(strategy)
            
            # Track voting
            if phase == 'voting':
                role = roles[current_idx]
                vote_stats[role]['total'] += 1
                if action == 1:
                    vote_stats[role]['ja'] += 1
                else:
                    vote_stats[role]['nein'] += 1
                
                # Check if voting for Hitler as chancellor
                if env.chancellor_nominee == hitler_idx:
                    if action == 1:
                        hitler_chanc_votes[role]['ja'] += 1
                    else:
                        hitler_chanc_votes[role]['nein'] += 1
            
            env.step(action)
    
    print("=" * 50)
    print("VOTING PATTERNS")
    print("=" * 50)
    
    for role in [0, 1, 2]:
        stats = vote_stats[role]
        if stats['total'] > 0:
            ja_rate = stats['ja'] / stats['total'] * 100
            print(f"{ROLE_NAMES[role]:10s}: {ja_rate:.1f}% JA ({stats['ja']}/{stats['total']})")
    
    print()
    print("Votes when HITLER is Chancellor nominee:")
    for role in [0, 1, 2]:
        stats = hitler_chanc_votes[role]
        total = stats['ja'] + stats['nein']
        if total > 0:
            ja_rate = stats['ja'] / total * 100
            print(f"  {ROLE_NAMES[role]:10s}: {ja_rate:.1f}% JA ({stats['ja']}/{total})")
    
    return vote_stats, hitler_chanc_votes

In [ ]:
vote_stats, hitler_votes = analyze_voting_strategies(cfr_agent, num_games=200)

## Analyze card selection strategies

In [ ]:
def analyze_card_strategies(cfr_agent, num_games=200):
    """Analyze how different roles select cards."""
    
    # Track: role -> phase -> {kept_liberal, kept_fascist, had_choice}
    card_stats = {
        role: {
            'prez': {'kept_lib': 0, 'kept_fasc': 0, 'had_choice': 0},
            'chanc': {'kept_lib': 0, 'kept_fasc': 0, 'had_choice': 0}
        }
        for role in [0, 1, 2]
    }
    
    for game_idx in range(num_games):
        env = ShitlerEnv()
        env.reset()
        roles = env.roles
        
        while not all(env.terminations.values()):
            current_agent = env.agent_selection
            current_idx = env.agents.index(current_agent)
            obs = env.observe(current_agent)
            phase = env.phase
            
            legal_actions = get_legal_actions(env, current_agent)
            if not legal_actions:
                env.step(0)
                continue
            
            infoset_key = get_infoset_key(obs, phase, current_idx)
            strategy = cfr_agent.get_average_strategy(infoset_key, legal_actions)
            action = cfr_agent.sample_action(strategy)
            
            # Track card selection
            if phase in ['prez_cardsel', 'chanc_cardsel']:
                role = roles[current_idx]
                cards = obs.get('cards', [])
                
                # Check if they had a choice (mixed cards)
                has_lib = 0 in cards
                has_fasc = 1 in cards
                
                if has_lib and has_fasc:
                    phase_key = 'prez' if phase == 'prez_cardsel' else 'chanc'
                    card_stats[role][phase_key]['had_choice'] += 1
                    
                    # What did they discard?
                    discarded = cards[action]
                    if discarded == 0:  # Discarded liberal = kept fascist
                        card_stats[role][phase_key]['kept_fasc'] += 1
                    else:  # Discarded fascist = kept liberal
                        card_stats[role][phase_key]['kept_lib'] += 1
            
            env.step(action)
    
    print("=" * 60)
    print("CARD SELECTION (when they had a choice)")
    print("=" * 60)
    
    for role in [0, 1, 2]:
        print(f"\n{ROLE_NAMES[role]}:")
        for phase_key, phase_name in [('prez', 'President'), ('chanc', 'Chancellor')]:
            stats = card_stats[role][phase_key]
            if stats['had_choice'] > 0:
                lib_rate = stats['kept_lib'] / stats['had_choice'] * 100
                print(f"  {phase_name:12s}: {lib_rate:.1f}% kept Liberal ({stats['kept_lib']}/{stats['had_choice']})")
            else:
                print(f"  {phase_name:12s}: No choices recorded")
    
    return card_stats

In [ ]:
card_stats = analyze_card_strategies(cfr_agent, num_games=200)

## Analyze claim strategies

In [ ]:
def analyze_claim_strategies(cfr_agent, num_games=200):
    """Analyze how different roles claim vs what they actually had."""
    
    # Track: role -> {truthful, lied_up, lied_down}
    claim_stats = {
        role: {
            'prez': {'truthful': 0, 'lied_up': 0, 'lied_down': 0, 'total': 0},
            'chanc': {'truthful': 0, 'lied_up': 0, 'lied_down': 0, 'total': 0}
        }
        for role in [0, 1, 2]
    }
    
    for game_idx in range(num_games):
        env = ShitlerEnv()
        env.reset()
        roles = env.roles
        
        # Track what cards were actually seen
        last_prez_cards = None
        last_chanc_cards = None
        
        while not all(env.terminations.values()):
            current_agent = env.agent_selection
            current_idx = env.agents.index(current_agent)
            obs = env.observe(current_agent)
            phase = env.phase
            
            legal_actions = get_legal_actions(env, current_agent)
            if not legal_actions:
                env.step(0)
                continue
            
            infoset_key = get_infoset_key(obs, phase, current_idx)
            strategy = cfr_agent.get_average_strategy(infoset_key, legal_actions)
            action = cfr_agent.sample_action(strategy)
            
            # Track cards seen
            if phase == 'prez_cardsel':
                last_prez_cards = obs.get('cards', []).copy()
            elif phase == 'chanc_cardsel':
                last_chanc_cards = obs.get('cards', []).copy()
            
            # Track claims
            if phase == 'prez_claim' and last_prez_cards is not None:
                role = roles[current_idx]
                actual_fasc = sum(1 for c in last_prez_cards if c == 1)
                claimed_fasc = action
                
                claim_stats[role]['prez']['total'] += 1
                if claimed_fasc == actual_fasc:
                    claim_stats[role]['prez']['truthful'] += 1
                elif claimed_fasc > actual_fasc:
                    claim_stats[role]['prez']['lied_up'] += 1
                else:
                    claim_stats[role]['prez']['lied_down'] += 1
                    
            elif phase == 'chanc_claim' and last_chanc_cards is not None:
                role = roles[current_idx]
                actual_fasc = sum(1 for c in last_chanc_cards if c == 1)
                claimed_fasc = action
                
                claim_stats[role]['chanc']['total'] += 1
                if claimed_fasc == actual_fasc:
                    claim_stats[role]['chanc']['truthful'] += 1
                elif claimed_fasc > actual_fasc:
                    claim_stats[role]['chanc']['lied_up'] += 1
                else:
                    claim_stats[role]['chanc']['lied_down'] += 1
            
            env.step(action)
    
    print("=" * 60)
    print("CLAIM HONESTY")
    print("=" * 60)
    
    for role in [0, 1, 2]:
        print(f"\n{ROLE_NAMES[role]}:")
        for phase_key, phase_name in [('prez', 'President'), ('chanc', 'Chancellor')]:
            stats = claim_stats[role][phase_key]
            if stats['total'] > 0:
                truth_rate = stats['truthful'] / stats['total'] * 100
                up_rate = stats['lied_up'] / stats['total'] * 100
                down_rate = stats['lied_down'] / stats['total'] * 100
                print(f"  {phase_name:12s}: {truth_rate:.1f}% truthful, {up_rate:.1f}% lied up (more fasc), {down_rate:.1f}% lied down")
    
    return claim_stats

In [ ]:
claim_stats = analyze_claim_strategies(cfr_agent, num_games=200)

## Look at specific infoset strategies

In [ ]:
def examine_early_game_strategies(cfr_agent):
    """Look at strategies in common early-game situations."""
    
    print("=" * 60)
    print("EARLY GAME STRATEGIES (first vote, no history)")
    print("=" * 60)
    
    # Simulate early game state
    env = ShitlerEnv()
    env.reset()
    
    # Find a voting phase
    while env.phase != 'voting' and not all(env.terminations.values()):
        current_agent = env.agent_selection
        legal_actions = get_legal_actions(env, current_agent)
        if legal_actions:
            env.step(legal_actions[0])
        else:
            env.step(0)
    
    if env.phase == 'voting':
        print(f"\nPresident: {env.president_idx}, Chancellor nominee: {env.chancellor_nominee}")
        print(f"Roles: {[ROLE_NAMES[r] for r in env.roles]}")
        print()
        
        for player_idx in range(5):
            agent = f'player_{player_idx}'
            obs = env.observe(agent)
            role = env.roles[player_idx]
            
            infoset_key = get_infoset_key(obs, 'voting', player_idx)
            legal_actions = [0, 1]  # NEIN, JA
            strategy = cfr_agent.get_average_strategy(infoset_key, legal_actions)
            
            print(f"Player {player_idx} ({ROLE_NAMES[role]}):")
            print(f"  JA prob: {strategy.get(1, 0):.1%}, NEIN prob: {strategy.get(0, 0):.1%}")
            
            # Check if this infoset was visited
            if infoset_key in cfr_agent.strategy_sums:
                total_visits = sum(cfr_agent.strategy_sums[infoset_key].values())
                print(f"  (Infoset visited ~{total_visits:.0f} times)")
            else:
                print(f"  (Infoset NOT in training data - using uniform)")

In [ ]:
examine_early_game_strategies(cfr_agent)

## Check infoset coverage

In [ ]:
def check_infoset_coverage(cfr_agent, num_games=100):
    """Check how often we encounter unseen infosets."""
    
    seen = 0
    unseen = 0
    unseen_by_phase = defaultdict(int)
    
    for _ in range(num_games):
        env = ShitlerEnv()
        env.reset()
        
        while not all(env.terminations.values()):
            current_agent = env.agent_selection
            current_idx = env.agents.index(current_agent)
            obs = env.observe(current_agent)
            phase = env.phase
            
            legal_actions = get_legal_actions(env, current_agent)
            if not legal_actions:
                env.step(0)
                continue
            
            infoset_key = get_infoset_key(obs, phase, current_idx)
            
            if infoset_key in cfr_agent.strategy_sums:
                seen += 1
            else:
                unseen += 1
                unseen_by_phase[phase] += 1
            
            strategy = cfr_agent.get_average_strategy(infoset_key, legal_actions)
            action = cfr_agent.sample_action(strategy)
            env.step(action)
    
    total = seen + unseen
    print(f"Infoset coverage: {seen}/{total} ({seen/total*100:.1f}%) seen during training")
    print(f"Unseen infosets by phase:")
    for phase, count in sorted(unseen_by_phase.items(), key=lambda x: -x[1]):
        print(f"  {phase}: {count}")

In [ ]:
check_infoset_coverage(cfr_agent, num_games=100)

## Summary statistics

In [ ]:
# Count infosets by phase
phase_counts = defaultdict(int)
for key in cfr_agent.regret_sums.keys():
    # Phase is at index 4 in the key tuple
    if len(key) > 4:
        phase = key[4]
        phase_counts[phase] += 1

print("Infosets by phase:")
for phase, count in sorted(phase_counts.items(), key=lambda x: -x[1]):
    print(f"  {phase}: {count:,}")

In [ ]:
# Count infosets by role
role_counts = defaultdict(int)
for key in cfr_agent.regret_sums.keys():
    # Role is at index 0 in the key tuple
    if len(key) > 0:
        role = key[0]
        role_counts[role] += 1

print("Infosets by role:")
for role, count in sorted(role_counts.items()):
    print(f"  {ROLE_NAMES.get(role, role)}: {count:,}")